In [ ]:
import tensorflow as tf
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import tensorflow.keras.backend as K
from tensorflow.keras import datasets, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, concatenate, Concatenate, UpSampling2D, Conv2DTranspose, Dense
from PIL import Image
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from datetime import datetime
import os
import matplotlib.pyplot as plt
import numpy as np
import glob

# Load the TensorBoard notebook extension
%load_ext tensorboard

data_dir = "H:\\COMP3710\\data"
dst_case_train = data_dir + "\\train"
dst_case_test = data_dir + "\\test"
dst_case_validate = data_dir + "\\validate"
dst_seg_train = data_dir + "\\train_seg"
dst_seg_test = data_dir + "\\test_seg"
dst_seg_validate = data_dir + "\\validate_seg"


In [ ]:
class CustomGenerator(keras.utils.Sequence):
    """Helper to iterate over the data (as Numpy arrays)."""

    def __init__(self, batch_size, img_size, input_img_paths, target_img_paths):
        self.batch_size = batch_size
        self.img_size = img_size
        self.input_img_paths = input_img_paths
        self.target_img_paths = target_img_paths

    def __len__(self):
        return len(self.target_img_paths) // self.batch_size

    def __getitem__(self, idx):
        """Returns tuple (input, target) correspond to batch #idx."""
        i = idx * self.batch_size
        batch_input_img_paths = self.input_img_paths[i : i + self.batch_size]
        batch_target_img_paths = self.target_img_paths[i : i + self.batch_size]
        x = np.zeros((batch_size,) + self.img_size + (3,), dtype="float32")
        for j, path in enumerate(batch_input_img_paths):
            img = load_img(path, target_size=self.img_size)
            x[j] = img
        y = np.zeros((batch_size,) + self.img_size + (1,), dtype="uint8")
        for j, path in enumerate(batch_target_img_paths):
            img = load_img(path, target_size=self.img_size, color_mode="grayscale")
            y[j] = np.expand_dims(img, 2)
        return x, y

In [ ]:

train_generator_images = image_data_generator_train.flow_from_dataframe(
    dataframe=train,
    directory='..//VOCdevkit/VOC2009/JPEGImages',
    x_col='filename',
    class_mode=None,
    color_mode="rgb",
    target_size=(image_size[1],image_size[0]),
    batch_size=batchSize,
    seed=seed)

train_generator_mask = mask_data_generator_train.flow_from_dataframe(
    dataframe=train,
    directory='..//VOCdevkit/VOC2009/SegmentationClass',
    x_col='segmentation',
    class_mode=None,
    color_mode="grayscale",
    target_size=(image_size[1],image_size[0]),
    batch_size=batchSize,
    seed=seed)

In [ ]:
## Load functions

def load_image(img_path):
    return np.array(Image.open(fname)) / 255.

def load_image_one_hot(img_path):
    img =  np.array(Image.open(fname)) * (1./255.) # 0 to 1
    img = np.ndarray.astype(img, np.uint8)
    return (np.arange(img.max()+1) == img[...,None]).astype(np.uint8)


In [4]:
## Load datasets

data_dir = "./"
folders = ["ISIC2018_Task1-2_Training_Input_x2/",
          "ISIC2018_Task1_Training_GroundTruth_x2/"]
case_pattern = "ISIC_???????.jpg"
seg_pattern = "ISIC_???????_segmentation.png"

# Pre-defined
h, w = 1296, 1936
seed = 123

case_file_pattern = os.path.join(data_dir, folders[0], case_pattern)
seg_file_pattern = os.path.join(data_dir, folders[1], seg_pattern)

case_fname = glob.glob(case_file_pattern)
seg_fname = glob.glob(seg_file_pattern)



In [5]:
print("cases: ", len(case_fname))
print("segs: ", len(seg_fname))
print(case_file_pattern)
print(seg_file_pattern)
print(os.getcwd())

cases:  2594
segs:  2594
./ISIC2018_Task1-2_Training_Input_x2/ISIC_???????.jpg
./ISIC2018_Task1_Training_GroundTruth_x2/ISIC_???????_segmentation.png
D:\s4354061


In [6]:
print(case_fname[0])
print(seg_fname[0])

./ISIC2018_Task1-2_Training_Input_x2\ISIC_0000000.jpg
./ISIC2018_Task1_Training_GroundTruth_x2\ISIC_0000000_segmentation.png


In [ ]:
## DATA FORMATTING
train_frac = 0.85
case, seg = [], []

print("Reading case...")
i=0
for fname in (case_fname):
    case.append(load_image(fname))
    print(i)
    i+=1

# train_oh = load_image_one_hot(tf.constant(train_fname))
    
print("Reading seg...")
i = 0
for fname in (seg_fname):
    seg.append(load_image_one_hot(fname))
    print(i)
    i+=1

train = np.array(train)
seg = np.array(seg)

case = case[:, :, :, np.newaxis]
seg = seg[:, :, :, np.newaxis]



print("Case images: ", case.shape, "\tdtype: ", case.dtype)
print("Seg images: ", seg.shape, "\tdtype: ", seg.dtype)

# del train_fname, test_fname, validate_fname

Reading case...


In [ ]:

case_shuffler = np. random. permutation(len(case))
seg_shuffler = np. random. permutation(len(seg))

case = case[case_shuffler]
seg = seg[seg_shuffler]